In [10]:
# Initial package imports
import json
import math
import pandas as pd
import psycopg2
from PIL import Image
import boto3
import numpy as np
import os
from dotenv import load_dotenv
from pascal_voc_writer import Writer

load_dotenv(dotenv_path="../.env")
S3_BUCKET = os.getenv('AWS_S3_BUCKET_NAME')
SRC_IMG_FOLDER = 'test'
DB_NAME = os.getenv("DB_NAME")
DB_HOST = os.getenv("DB_HOST")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD =  '2yG5$A#LkJkvnWh*' #os.getenv("DB_PASSWORD")

In [3]:
# SQL queries to the database
def queryDB(query):
    conn = psycopg2.connect(database = DB_NAME,
                        user = DB_USER,
                        password = DB_PASSWORD,
                        host = DB_HOST,
                        port = "5432")
    cur = conn.cursor()
    result = pd.read_sql_query(query, conn)
    conn.close()
    return result

In [4]:
annotations = queryDB("select * from annotations limit 100")
annotations.head()

,id,videoid,userid,conceptid,timeinvideo,x1,y1,x2,y2,videowidth,videoheight,dateannotated,image,imagewithbox,comment,unsure,originalid
0,917345,37,17,236,384.496273,294.0,348.0,305.0,359.0,640.0,360.0,2018-12-21,873945_384.4962733333333_ai.png,873945_384.4962733333333_box_ai.png,,False,873945
1,917352,37,17,236,384.563007,294.0,348.0,305.0,359.0,640.0,360.0,2018-12-21,873945_384.56300666666664_ai.png,873945_384.56300666666664_box_ai.png,,False,873945
2,962445,13,17,2136,437.620313,265.0,226.0,322.0,264.0,640.0,360.0,2018-12-22,1808_437.62031333333334_ai.png,1808_437.62031333333334_box_ai.png,None,False,1808
3,962447,13,17,2136,437.553580,265.0,221.0,322.0,259.0,640.0,360.0,2018-12-22,1808_437.55358_ai.png,1808_437.55358_box_ai.png,None,False,1808
4,962450,13,17,2136,437.687047,265.0,226.0,322.0,264.0,640.0,360.0,2018-12-22,1808_437.6870466666667_ai.png,1808_437.6870466666667_box_ai.png,None,False,1808


s3fs instead of download data?

In [21]:
def format_annotations(annotations, split=.8, img_folder='test'):
    annotations.sample(frac=1)
    count = 0
    folder = 'train'
    for name,group in annotations.groupby(['videoid','timeinvideo']):
        #download image or SOMETHING
        first = group.iloc[0]
        writer = Writer(img_folder + '/' + first['image'], first['videowidth'], first['videoheight'])

        for index, row in group.iterrows():
            writer.addObject(row['conceptid'], row['x1'], row['y1'], row['x2'], row['y2'])

        if(count > len(annotations) * split):
            folder = 'test'
        count += 1
        writer.save(folder + '/' + first['image'])
format_annotations(annotations)


100